# Proyecto Final - Aprendizaje Automático - Diego Estrada

## Problema de interés de analisis: Calidad de agua en el Río de la Plata

In [26]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

#Definimos una ruta base absoluta para levantar los 6 dataset
ruta_base = 'C:\\Users\\destrada\\Ciencia_datos_2A1C\\Cookiecutter_Proyecto_Final\\data\\'

#Verificamos si la ruta existe para evitar falla de localizacion de archivos, con la condicion IF validamos la existencia de ruta.
if os.path.exists(ruta_base):
    # Cargamos los datasets
    df1 = pd.read_csv(ruta_base + 'agc_y_riodelaplata_2017.csv', encoding='latin1', sep=';')
    df2 = pd.read_csv(ruta_base + 'agc_z_riodelaplata_2018.csv', encoding='latin1', sep=';')
    df3 = pd.read_csv(ruta_base + 'agc_z_riodelaplata_2019.csv', encoding='latin1', sep=';')
    df4 = pd.read_csv(ruta_base + 'agc_z_riodelaplata_2020.csv', encoding='latin1', sep=';')
    df5 = pd.read_csv(ruta_base + 'agc_z_riodelaplata_2021.csv', encoding='latin1', sep=';')
    df6 = pd.read_csv(ruta_base + 'agc_y_riodelaplata_2022.csv', encoding='latin1', sep=';')

    #Concatenamos los dataframes en uno solo Dataframe
    df_final = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)
else:
    #En caso de no encontrar los archivos en la ruta, mostraria este mensaje.
    print("La ruta especificada no existe.")
    
#Mostramos las primeras filas del DataFrame unificado
df_final.head()


,Orden,Sitios,Gobierno_local,codigo,Fecha,Año,Campaña,Tem_agua,OD,pH,...,enteroc_ufc_100ml,P_de_ortof_mg_l,Hidr_Deriv_del_Petroleo_ug_l,Microcistina_ug_l,Clorofila_a_ug_l,ICA,Calidad_de_agua,Tem_aire,escherichia_coli_ufc_100ml,Hidrocarburos_Deriv_del_Petróleo_ug_l
0,1,Canal Villanueva y Río Luján,Tigre,TI001,8/3/2017,2017,verano,no se midió,6.01,7.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Río Lujan y Arroyo Caraguatá,Tigre,TI006,8/3/2017,2017,verano,no se midió,6.54,7.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Canal Aliviador y Río Lujan,Tigre,TI002,8/3/2017,2017,verano,no se midió,4.8,7.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Río Carapachay y Arroyo Gallo Fiambre,Tigre,TI003,8/3/2017,2017,verano,no se midió,5.89,7.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Río Reconquista y Río Lujan,Tigre,TI004,8/3/2017,2017,verano,no se midió,4.5,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
#Queremos conocer la cantidad de registros en el DataFrame Unificado
num_registros = df_final.shape[0]
print("Cantidad de registros: " + str(num_registros))

#Cantidad de columnas
num_columnas = df_final.shape[1]
print("Cantidad de columnas: " + str(num_columnas))

Cantidad de registros: 902
Cantidad de columnas: 39


##  Preprocesamiento de Datos (Data Preprocessing)
Vamos a trabajar en el proceso de ETL, después que unificamos los datos, eliminaremos algunas columnas que no necesitamos analizar, lo que también mejorará el rendimiento del modelo que vamos a desarrollar.

In [31]:
#Estantarizamos los nombre de las columnas
df_final.columns = df_final.columns.str.lower()

orden 